In [12]:
pip install requests faiss-cpu sentence-transformers langchain PyPDF2 numpy

Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import json
import numpy as np
import faiss
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

### KONFIGURASI DASAR OPENROUTER + GLM

In [12]:
OPENROUTER_KEY = "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"
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "z-ai/glm-4.5"

### PEMANGGIL MODEL GLM VIA OPENROUTER

In [26]:
def call_glm(prompt: str, temperature: float = 0.0) -> str:
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Kamu adalah sistem penilai otomatis. Jawab hanya dalam format JSON."},
            {"role": "user", "content": prompt},
        ],
        "temperature": temperature,
    }

    try:
        r = requests.post(OPENROUTER_URL, headers=headers, json=payload, timeout=120)
        r.raise_for_status()
        data = r.json()
        return data["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"⚠️ Gagal memanggil GLM: {e}")
        print("Respons mentah:", r.text if 'r' in locals() else "tidak ada respons")
        return "{}"

### LOAD RUBRIK PENILAIAN (JSON)

In [27]:
def load_rubric_json(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        rubric_data = json.load(f)
    return json.dumps(rubric_data, indent=2, ensure_ascii=False)



### EKSTRAK TEKS DARI PDF MAHASISWA

In [28]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    pages = []
    for page in reader.pages:
        p = page.extract_text() or ""
        if p.strip():
            pages.append(p)
    return "\n".join(pages)

### chunk text

In [29]:
def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    if not text or not text.strip():
        return []
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return [c.strip() for c in chunks if c and c.strip()]

### encoding ke Vector DB

In [30]:
class SimpleVectorDB:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.index = None
        self.texts = []

    def build(self, texts):
        self.texts = texts
        embeddings = self.embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)
        embeddings = np.asarray(embeddings, dtype=np.float32)
        dim = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(embeddings)

    def search(self, query, k=3):
        q_emb = self.embedder.encode([query], convert_to_numpy=True)
        q_emb = np.asarray(q_emb, dtype=np.float32)
        D, I = self.index.search(q_emb, k)
        return [self.texts[i] for i in I[0]]

### Build Promt

In [31]:
def build_prompt_with_rag(rubric_json: str, evidence: str) -> str:
    return f"""
Anda adalah sistem auto-grading untuk Learning Management System (LMS).
Tugas Anda adalah menilai laporan mahasiswa berdasarkan **rubrik penilaian (JSON)** dan **evidence dari hasil pencarian dokumen (RAG)**.

Gunakan *hanya informasi dari evidence di bawah ini* sebagai dasar penilaian.
JANGAN mengarang isi di luar evidence.

---

### 📘 RUBRIK PENILAIAN (JSON)
{rubric_json}

---

### 📄 EVIDENCE DARI LAPORAN (hasil pencarian RAG)
Gunakan bagian-bagian teks berikut untuk mendukung setiap penilaian:
{evidence}

---

### 📋 INSTRUKSI PENILAIAN
1. Evaluasi **SEMUA sub-rubrik** yang ada pada rubrik JSON, jangan ada yang dilewatkan.
2. Setiap level memiliki rentang nilai (`score_range`). Pilih **nilai numerik** dalam rentang itu, bukan hanya ujungnya.
   - Contoh: jika sesuai level B (61–80), boleh kasih 70 atau 75 tergantung kualitas.
   - Jika isi tidak mencukupi, tetap tampilkan sub-rubrik tersebut dengan level terendah dan beri alasan.
3. Gunakan bukti dari bagian *Evidence* di atas untuk setiap keputusan penilaian.
   - Jika ada kalimat pendukung, sebutkan ringkas potongan teks evidence yang relevan.
4. Sertakan alasan singkat (1–3 kalimat) yang berdasarkan evidence.
5. Cantumkan bobot (`assignment_sub_rubrics.weight`).
6. Hitung nilai total berdasarkan skor × bobot.
7. Tambahkan confidence score untuk setiap sub-rubrik dan keseluruhan (`overall_confidence`).

---

### 📤 FORMAT OUTPUT WAJIB (JSON)
Jawaban akhir **HARUS** mengikuti struktur JSON berikut:

{{
  "grading_result": [
    {{
      "sub_rubric": "Nama Sub Rubrik",
      "selected_level": "A/B/C/...",
      "score_awarded": 0-100,
      "weight": 0-100,
      "reason": "alasan singkat berdasarkan evidence",
      "evidence_quote": "potongan teks relevan dari evidence",
      "confidence": 0.0-1.0
    }}
  ],
  "final_score": 0-100,
  "overall_confidence": 0.0-1.0
}}

Output tidak boleh berisi penjelasan tambahan di luar struktur JSON.
"""


In [34]:
if __name__ == "__main__":
    rubric_file = "rubrik.JSON"
    pdf_file = "Laprak _Minggu 2 dan 3_Kelompok 6.pdf"

    print("Memuat rubrik penilaian...")
    rubric_json = load_rubric_json(rubric_file)

    print("Mengekstrak teks dari PDF...")
    pdf_text = extract_text_from_pdf(pdf_file)
    if not pdf_text:
        raise SystemExit("PDF kosong (kemungkinan hasil scan)")

    print("Memecah teks menjadi potongan kecil...")
    chunks = chunk_text(pdf_text)
    if not chunks:
        chunks = [pdf_text[:20000]]

    print("Membangun database vektor (FAISS)...")
    vectordb = SimpleVectorDB()
    vectordb.build(chunks)

    print("Mencari evidence relevan...")
    query = "Evaluasi laporan sesuai rubrik penilaian. Cari bukti tujuan penelitian, metode, dan hasil."
    evidence_chunks = vectordb.search(query, k=5)
    evidence = "\n\n---\n\n".join(evidence_chunks)

    print("Menyusun prompt untuk GLM...")
    prompt = build_prompt_with_rag(rubric_json, evidence)

    print("Mengirim prompt ke GLM melalui OpenRouter...")
    response = call_glm(prompt)

    print("\n>>> HASIL PENILAIAN (JSON):")
    print(response)

Memuat rubrik penilaian...
Mengekstrak teks dari PDF...
Memecah teks menjadi potongan kecil...
Membangun database vektor (FAISS)...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Mencari evidence relevan...
Menyusun prompt untuk GLM...
Mengirim prompt ke GLM melalui OpenRouter...

>>> HASIL PENILAIAN (JSON):
```json
{
  "grading_result": [
    {
      "sub_rubric": "Dasar Teori",
      "selected_level": "A",
      "score_awarded": 90,
      "weight": 10,
      "reason": "Dasar teori dikemukakan dengan jelas dan informatif, mencakup penjelasan tentang variabel, operator aritmatika, dan konversi tipe data dengan definisi yang tepat.",
      "evidence_quote": "Variabel merupakan suatu tempat yang tersedia di memori komputer yang berguna untuk menyimpan data baik itu huruf, rangkaian huruf (ekuivalen dengan kata atau kalimat), angka (bilangan bulat/desimal), atau karakter khusus.",
      "confidence": 0.9
    },
    {
      "sub_rubric": "Kode Program",
      "selected_level": "B",
      "score_awarded": 70,
      "weight": 50,
      "reason": "Evidence hanya menunjukkan potongan kode terbatas tanpa menampilkan keseluruhan program atau informasi jelas tentang efisi